In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os
import numpy as np
import pandas as pd
import sys
import cv2


In [2]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")
print('Device used:', device)

Device used: cuda


In [3]:
class FaceImage(Dataset):
    def __init__(self,label,picture_file,path,Train=True):
        self.data_path = path
        self.label = label
        self.train = Train
        self.picture_file = picture_file
    def __getitem__(self,index):

        file_path = os.path.join(self.data_path,self.picture_file[index])
        img = cv2.imread(file_path,cv2.IMREAD_GRAYSCALE)
        img = np.expand_dims(img,0)
        img =torch.tensor(img).float()
        if self.train==True:
            return torch.FloatTensor(img),self.label[index,1]
        else:
            return img

    def __len__(self):
        return len(os.listdir(self.data_path))


In [4]:
class ImageNet(nn.Module):
    def __init__(self):
        super(ImageNet,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 48, kernel_size=3,stride =(1,1)),
            nn.Dropout2d(0.3),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(48, 128, kernel_size=3,stride =(1,1)),
            nn.Dropout2d(0.4),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3,stride =(1,1)),
            nn.Dropout2d(0.4),
            nn.MaxPool2d(2),
            nn.ReLU()
        )

        self.fcn1 = nn.Linear(in_features =4096 ,out_features =512 ,bias = True)
        self.fcn2 = nn.Linear(in_features = 512,out_features =256 ,bias = True)
        self.fcn3 = nn.Linear(in_features =256 ,out_features =7 ,bias = True)

    def forward(self , x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0),-1)
        #print("size:",x.shape)
        x = F.relu(self.fcn1(x))
        x = F.relu(self.fcn2(x))
        out = self.fcn3(x)
        
        return out

In [5]:
train_label = pd.read_csv('./train.csv')
train_label = np.array(train_label,dtype = int)
datalen  = len(train_label)


picture_file = sorted(os.listdir('./train_img'))
train_dataset = FaceImage(train_label,picture_file,'./train_img')
picture_file = sorted(os.listdir('./test_img'))
test_dataset = FaceImage(train_label,picture_file,'./test_img',Train = False)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 256)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size = 256)


In [6]:
#model = ImageNet().cuda()
model = torch.load("Imagenet3.pkl")
print(model)
for epoch in range(800):
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    model.train()
    train_loss = 0
    correct = 0

    for batch_index ,(data, label) in enumerate(train_loader):
        data, label =data.cuda(), label.cuda()
        optimizer.zero_grad()
        output = model(data)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_index * len(data), len(train_loader.dataset),100. * batch_index / len(train_loader), loss.item()),end = "\r")
    if epoch%100==0:
        torch.save(model,"Imagenet4.pkl")


ImageNet(
  (conv1): Sequential(
    (0): Conv2d(1, 48, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.4, inplace=False)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(48, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.3, inplace=False)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (1): Dropout2d(p=0.3, inplace=False)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
  )
  (fcn1): Linear(in_features=4096, out_features=512, bias=True)
  (fcn2): Linear(in_features=512, out_features=256, bias=True)
  (fcn3): Linear(in_features=256, out_features=7, bias=True)
)


/home/haoxiang/miniconda3/envs/ml2019fall_hw3/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type ImageNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [7]:
torch.save(model,"Imagenet4.pkl")

In [ ]:
#model = torch.load("Imagenet.pkl",map_location = "cpu")
model.eval()
model.cpu()
ans = []
for idx,test in enumerate(test_loader):
    test =test
    ans.append(model(test))


In [ ]:
ans_final =[]
for a in ans:
    for pre in a:
        pre = pre.tolist()
        ans_final.append(pre.inedx(max(pre)))

In [ ]:
with open("cnn3.csv","w") as f:
    print("id,label", file = f)
    for id,label in enumerate(ans_final):
        print("{},{}".format(id,label) ,file = f)

# Pytorch 注意事項

* pytorch 不用在output layer加 softmax 因為loss 裡面包好了
* Pytorch 不用one-hot-encoding 額外寫會出錯



